# Dual numbers

## definition
Like imaginary numbers, a dual number is pair of real numbers with the defined constant $\epsilon$ such that $\epsilon^2 = 0$. So, $\forall a,b \in \mathbb{R}$,
$$(a+b\epsilon)(a-b\epsilon) = a^2$$
and
$$(a+b\epsilon)^n = a^n + na^{n-1}b\epsilon$$
where the second property can be easily proved by induction,
$$
\begin{aligned}
(a+b\epsilon)^{n+1}
&=
(a+b\epsilon) (a+b\epsilon)^n
\\&=
(a+b\epsilon) \left(a^n + na^{n-1}b\epsilon\right)
\\&=
a^{n+1} + na^n b\epsilon + a^nb\epsilon
\\&=
a^{n+1} + (n+1)a^n b\epsilon .
\end{aligned}
$$

Considering $P(x) = \sum_n a_n x^n$, then
$$
\begin{aligned}
P(x+h\epsilon) 
&=
\sum_n a_n (x+h\epsilon)^n
\\&=
\sum_n a_n x^n + \sum_n n a_n x^{n-1} h\epsilon
\\&=
P(x) + P'(x)h\epsilon .
\end{aligned}
$$
So, for any analytical function $f$,
$$f(x+h\epsilon) = f(x) + f'(x)h\epsilon.$$

Let $f$ and $g$ be two analytical functions, the arithmetic operations plus the function composition of $f$ and $g$ applied to a dual number correspond to all the derivative rules including the chain rule,
$$
\begin{aligned}
f(x+h\epsilon) \pm g(x+h\epsilon) 
&= f(x) + f'(x)h\epsilon \pm g(x) \pm g'(x)h\epsilon
\\&= \left( f(x) \pm g(x) \right) + \left( f'(x) \pm g'(x) \right)h\epsilon ,
\\[1em]
f(x+h\epsilon) \times g(x+h\epsilon) 
&= \left( f(x) + f'(x)h\epsilon \right) \times \left( g(x) \pm g'(x)h\epsilon \right)
\\&= f(x)g(x) + \left( f'(x)g(x) + f(x)g'(x) \right)h\epsilon ,
\\[1em]
\frac{f(x+h\epsilon)}{g(x+h\epsilon)}
&= \frac{f(x+h\epsilon)}{g(x+h\epsilon)} \times \frac{g(x-h\epsilon)}{g(x-h\epsilon)}
\\&= \frac{f(x)g(x) + \left( f'(x)g(x) - f(x)g'(x) \right)h\epsilon}{g(x)^2}
\\&= \frac{f(x)}{g(x)} + \frac{f'(x)g(x) - f(x)g'(x)}{g(x)^2}h\epsilon ,
\\[1em]
f(g(x+h\epsilon))
&= f(g(x) + g'(x)h\epsilon)
\\&= f(g(x)) + f'(g(x)) g'(x) h\epsilon .
\end{aligned}
$$

## implementing dual numbers in julia
We first start with a structure to associate $f$ and $f'$.

In [1]:
struct Dual <: Number
    x::Real
    dx::Real
end

Next, we define how to convert a variable $x$ into a dual number.

In [2]:
Dual(x) = Dual(x, one(x));

Then we define how to convert constant numbers into dual numbers.

In [3]:
Base.convert(::Type{Dual}, x::Real) = Dual(x, zero(x))

And how to typecast any real number into a dual number.

In [4]:
Base.promote_rule(::Type{Dual}, ::Type{<:Real}) = Dual

The next step is define the arithmetic properties.

In [5]:
Base.:+(f::Dual, g::Dual) = Dual(f.x + g.x, f.dx + g.dx)
Base.:-(f::Dual, g::Dual) = Dual(f.x - g.x, f.dx - g.dx)
Base.:*(f::Dual, g::Dual) = Dual(f.x * g.x, f.dx * g.x + f.x * g.dx)
Base.:/(f::Dual, g::Dual) = Dual(f.x / g.x, (f.dx * g.x - f.x * g.dx) / g.x^2)

To make things easy we can define the constant $\epsilon$.

In [6]:
const ϵ = Dual(0,1)
ϵ^2

Dual(0, 0)

We can make it more visual by define a print format.

In [7]:
Base.show(io::IO, f::Dual) = print(io, f.x, " + ", f.dx, "ϵ")

ϵ^2

0 + 0ϵ

Now, let's define how to use some standard functions with dual numbers. Thanks to the multiple dispatch we can add more functions to this list later.

In [8]:
Base.sin(f::Dual) = sin(f.x) + cos(f.x) * f.dx * ϵ
Base.cos(f::Dual) = cos(f.x) - sin(f.x) * f.dx * ϵ
Base.exp(f::Dual) = exp(f.x) + exp(f.x) * f.dx * ϵ
Base.log(f::Dual) = log(f.x) + f.dx / f.x * ϵ
Base.sqrt(f::Dual) = sqrt(f.x) + f.dx / sqrt(f.x) * ϵ

In [9]:
f(x) = x^2 + 2x - cos(sqrt(x))
f(Dual(π))

16.353083249392316 + 8.83594211490989ϵ